https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

In [10]:
import os
import csv
import datetime
import dateutil
import requests


#Settings

keyword = "#traveltuesday lang:en"
start_time = "2020-01-01T00:00:00.000Z" 
end_time = "2022-12-04T00:00:00.000Z"
max_results = 500
tweets= 2000

os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAHg%2BkAEAAAAApzRRZfeOzXkJCViljinNCRxvPA0%3DDTsCyNv33D9Bozvtg558p1YPN4UYzbT5Ne5HGJEqD2uHN2Ri8w'

In [11]:
def auth():
    return os.getenv('TOKEN')

In [12]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [13]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)

In [14]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [15]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [16]:
# Create file

def createFile():
    date = datetime.date.today().strftime("%m-%d-%Y")
    name = date + "data.csv"
    #csvFile = open(date +"data.csv", "a", newline="", encoding='utf-8')
    csvFile = open(name, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
    csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
    csvFile.close()
    return csvFile.name

In [17]:

def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [18]:

#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)

#Total number of tweets we collected from the loop
total_tweets = 0

    # Inputs
count = 0 # Counting tweets per time period
flag = True
next_token = None

csvFile = createFile()

# Check if flag is true
while flag:
    # Check if max_count reached
    if count >= tweets:
        break
    print("-------------------")
    print("Token: ", next_token)
    url = create_url(keyword, start_time,end_time, max_results)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)
        if result_count is not None and result_count > 0 and next_token is not None:
            print("Start Date: ", start_time)
            append_to_csv(json_response, csvFile)
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")              
    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            print("Start Date: ", start_time)
            append_to_csv(json_response, csvFile)
            count += result_count
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
        
        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzhmkvgp218soxts9rjal6lup3gjh
Start Date:  2020-01-01T00:00:00.000Z
# of Tweets added from this response:  499
Total # of Tweets added:  499
-------------------
-------------------
Token:  b26v89c19zqg8o3fpzhmkvgp218soxts9rjal6lup3gjh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzhm64qbxkdif9xsuyue9zrwr37gd
Start Date:  2020-01-01T00:00:00.000Z
# of Tweets added from this response:  499
Total # of Tweets added:  998
-------------------
-------------------
Token:  b26v89c19zqg8o3fpzhm64qbxkdif9xsuyue9zrwr37gd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzhm62lwkmq3gxkb3sv9fr6fmaqnx
Start Date:  2020-01-01T00:00:00.000Z
# of Tweets added from this response:  483
Total # of Tweets added:  1481
-------------------
-------------------
Token:  b26v89c19zqg8o3fpzhm62lwkmq3gxkb3sv9fr6fmaqnx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpzhm60hs06ku66vsdgluokj1yfg59
Sta